<a href="https://colab.research.google.com/github/MichaelArgs/Dog-Breed-Classification/blob/main/dog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Instal Streamlit
!pip install streamlit

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.9/41.9 kB 981.8 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.7/8.7 MB 47.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 78.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.3/79.3 kB 7.4 MB/s eta 0:00:00


In [2]:
# Import library yang dibutuhkan untuk deep learning
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, GlobalAveragePooling2D
import numpy as np
import streamlit as st
from PIL import Image
import zipfile
import json
from google.colab import files

In [3]:
# Unzip dataset yang berisi gambar jenis anjing
zip_path = '/content/70 dog breed.zip'
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall('/content/70 dog breed')

In [4]:
train_path = '/content/70 dog breed/train'
valid_path = '/content/70 dog breed/valid'
test_path = '/content/70 dog breed/test'

In [5]:
# Data Augmentation untuk memperbanyak variasi data pelatihan
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.3,
    horizontal_flip=True,
    fill_mode='nearest'
)

valid_datagen = ImageDataGenerator(rescale=1./255)

# Membuat generator data untuk pelatihan dan validasi
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical'
)

valid_generator = valid_datagen.flow_from_directory(
    valid_path,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical'
)

Found 7946 images belonging to 70 classes.
Found 700 images belonging to 70 classes.


In [6]:
# Pemanggilan model MobileNetV2 sebagai base model
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(128, 128, 3))

# Fine-tuning
for layer in base_model.layers[-50:]:
    layer.trainable = True

# Menambahkan lapisan tambahan di atas base model
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(train_generator.num_classes, activation='softmax')(x)

# Definisikan model lengkap dengan MobileNetV2 dan lapisan tambahan
model = Model(inputs=base_model.input, outputs=output)

# Kompilasi model dengan optimizer Adam dan learning rate kecil
model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])

9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [7]:
# EarlyStopping untuk menghentikan pelatihan jika tidak ada peningkatan dalam 5 epoch
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Mengurangi learning rate jika tidak ada peningkatan dalam 3 epoch
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6, verbose=1)

# Kumpulan callbacks
callbacks = [early_stopping, reduce_lr]

In [8]:
# Melatih model menggunakan generator data pelatihan dan validasi
history = model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=30,
    callbacks=callbacks
)

Epoch 1/30


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


249/249 ━━━━━━━━━━━━━━━━━━━━ 481s 2s/step - accuracy: 0.1324 - loss: 3.8695 - val_accuracy: 0.5829 - val_loss: 1.4975 - learning_rate: 1.0000e-04
Epoch 2/30
249/249 ━━━━━━━━━━━━━━━━━━━━ 494s 2s/step - accuracy: 0.5467 - loss: 1.6563 - val_accuracy: 0.7100 - val_loss: 1.1287 - learning_rate: 1.0000e-04
Epoch 3/30
249/249 ━━━━━━━━━━━━━━━━━━━━ 496s 2s/step - accuracy: 0.6720 - loss: 1.1742 - val_accuracy: 0.7343 - val_loss: 0.9943 - learning_rate: 1.0000e-04
Epoch 4/30
249/249 ━━━━━━━━━━━━━━━━━━━━ 420s 2s/step - accuracy: 0.7271 - loss: 0.9489 - val_accuracy: 0.7257 - val_loss: 1.1016 - learning_rate: 1.0000e-04
Epoch 5/30
249/249 ━━━━━━━━━━━━━━━━━━━━ 433s 2s/step - accuracy: 0.7562 - loss: 0.8219 - val_accuracy: 0.7471 - val_loss: 0.9776 - learning_rate: 1.0000e-04
Epoch 6/30
249/249 ━━━━━━━━━━━━━━━━━━━━ 410s 2s/step - accuracy: 0.7810 - loss: 0.7408 - val_accuracy: 0.7900 - val_loss: 0.8484 - learning_rate: 1.0000e-04
Epoch 7/30
249/249 ━━━━━━━━━━━━━━━━━━━━ 442s 2s/step - accuracy: 0.78

In [9]:
model.save("dog_breed_classifier.keras")

files.download("dog_breed_classifier.keras")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [10]:
breed_names = train_generator.class_indices
with open("class_indices.json", "w") as f:
    json.dump(breed_names, f)

files.download("class_indices.json")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>